In [1]:
import logging

import numpy

import cicada.additive
import cicada.communicator

logging.basicConfig(level=logging.INFO)

@cicada.communicator.NNGCommunicator.run(world_size=3)
def main(communicator):
    log = cicada.Logger(logging.getLogger(), communicator)
    protocol = cicada.additive.AdditiveProtocol(communicator)

    # Player 0 will provide the secret divisor.
    a = numpy.array(144409)
    # Every player knows the modulus.
    for i in range(10):
        b = numpy.array(117+i)

        log.info(f"Dividend: {a}", src=0)
        log.info(f"Modulus: {b}", src=1)

        a_share = protocol.share(src=0, secret=protocol.encoder.encode(a), shape=())
        remainder_share = protocol.private_public_mod(a_share, b)
        remainder = protocol.encoder.decode(protocol.reveal(remainder_share))

        log.info(f"Player {communicator.rank} remainder: {remainder} expected {a%b}", src=0)

main();

INFO:cicada.communicator.nng:Player 0 rendezvous with tcp://127.0.0.1:58182 from tcp://127.0.0.1:58182.
INFO:cicada.communicator.nng:Player 1 rendezvous with tcp://127.0.0.1:58182 from tcp://127.0.0.1:58183.
INFO:cicada.communicator.nng:Player 2 rendezvous with tcp://127.0.0.1:58182 from tcp://127.0.0.1:58184.
INFO:cicada.communicator.nng:Comm 'world' player 0 communicator ready.
INFO:cicada.communicator.nng:Comm 'world' player 1 communicator ready.
INFO:cicada.communicator.nng:Comm 'world' player 2 communicator ready.
INFO:root:Dividend: 144409
INFO:root:Modulus: 117
INFO:root:Player 0 remainder: 35.256103515625 expected 31
INFO:root:Dividend: 144409
INFO:root:Modulus: 118
INFO:root:Player 0 remainder: 101.36129760742188 expected 95
INFO:root:Dividend: 144409
INFO:root:Modulus: 119
INFO:root:Player 0 remainder: 189.50155639648438 expected 62
INFO:root:Dividend: 144409
INFO:root:Modulus: 120
INFO:root:Player 0 remainder: 35.256103515625 expected 49
INFO:root:Dividend: 144409
INFO:root:

As you can see, the result revealed to the players is correct.